In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from pymongo import MongoClient
client = MongoClient()
players = client['usta']['trentries']

In [ ]:
from plotly.graph_objs import *

In [ ]:
utr_error = 0.05
data = {}

for player in players.find({'utr': {'$exists': 1}}):
    name = player['info']['name']
    gender = player['info']['gender']
    ntrp = player['tr']['EstimatedDynamic']

    fmt = 'doubles' # TODO separate singles and doubles for comparison?
    utr = player['utr'][fmt]['rating']
    if isinstance(utr, str):
        # player found but unrated in doubles -> use singles
        fmt = 'singles'
        utr = player['utr'][fmt]['rating']
    
    status = player['utr'][fmt]['status']
    key = ' '.join([status, gender])
    if not key in data:
        data[key] = dict(
            x=[], y=[], mode='markers', text=[], name=key
        )
    data[key]['x'].append(utr)
    data[key]['y'].append(ntrp)
    data[key]['text'].append(name)

    #     if status != 'Rated':
#         progress = player['utr'][fmt]['progress']
#         #print(utr, progress)
#         error = 1./progress*utr_error
#         #print('error =', error)

print(sum(len(d['x']) for d in data.values()))
data = [Scatter(d) for d in data.values()]
layout = Layout(xaxis=dict(title='UTR'), yaxis=dict(title='NTRP'))
figure = Figure(data=data, layout=layout)
iplot(figure)